![](https://camo.githubusercontent.com/92de74138720e79fda16c575fac6677e6de9089b/68747470733a2f2f7773342e73696e61696d672e636e2f6c617267652f303036744e6337396c7931666d706b35736d7469646a3330376e3064783379762e6a7067)

In [1]:
import torch
import numpy as np
from torch import nn
from torch.autograd import Variable
from torchvision.datasets import CIFAR10

In [2]:
def vgg_block(num_convs, in_channels, out_channels):
    net = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), nn.ReLU(True)]  # first layer

    for i in range(num_convs - 1):
        net.append(nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1))
        net.append(nn.ReLU(True))

    net.append(nn.MaxPool2d(2, 2))
    return nn.Sequential(*net)

In [3]:
block_demo = vgg_block(3, 64, 128)
print(block_demo)


Sequential(
  (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace)
  (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace)
  (4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): ReLU(inplace)
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)


In [4]:
# 首先定义输入为 (1, 64, 300, 300)
input_demo = Variable(torch.zeros(1, 64, 300, 300))
output_demo = block_demo(input_demo)
print(output_demo.shape)

torch.Size([1, 128, 150, 150])


In [5]:
def vgg_stack(num_convs, chnnels):
    net = []
    for n, c in zip(num_convs, chnnels):
        in_c = c[0]
        out_c = c[1]
        net.append(vgg_block(n, in_c, out_c))
    return nn.Sequential(*net)

In [6]:
vgg_net = vgg_stack((1, 1, 2, 2, 2), ((3, 64), (64, 128), (128, 256), (256, 512), (512, 512)))
print(vgg_net)

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (2): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (3): Sequential(
    (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=

In [7]:
test_x = Variable(torch.zeros(1, 3, 256, 256))
test_y = vgg_net(test_x)
print(test_y.shape)

torch.Size([1, 512, 8, 8])


In [8]:
class Vgg(nn.Module):
    def __init__(self):
        super(Vgg, self).__init__()
        self.feature = vgg_net
        self.fc = nn.Sequential(
            nn.Linear(512, 100),
            nn.ReLU(True),
            nn.Linear(100, 10)
        )

    def forward(self, x):
        x = self.feature(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x


In [9]:
def data_tf(x):
    x = np.array(x, dtype='float32') / 255
    x = (x - 0.5) / 0.5  # 标准化，这个技巧之后会讲到
    x = x.transpose((2, 0, 1))  # 将 channel 放到第一维，只是 pytorch 要求的输入方式
    x = torch.from_numpy(x)
    return x


In [10]:
train_set = CIFAR10('./data', train=True, transform=data_tf)
train_data = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
test_set = CIFAR10('./data', train=False, transform=data_tf)
test_data = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False)

In [11]:
net = Vgg()
optimizer = torch.optim.SGD(net.parameters(), lr=1e-1)
criterion = nn.CrossEntropyLoss()

In [12]:
from utils import train

train(net, train_data, test_data, 20, optimizer, criterion)

D:\Workspace\Samples\pytorch_lab\chapter03_cnn\utils.py:54: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  im = Variable(im.cuda(), volatile=True)
D:\Workspace\Samples\pytorch_lab\chapter03_cnn\utils.py:55: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  label = Variable(label.cuda(), volatile=True)


Epoch 0. Train Loss: 2.303118, Train Acc: 0.099764, Valid Loss: 2.302887, Valid Acc: 0.100969, Time 00:00:45
Epoch 1. Train Loss: 2.303010, Train Acc: 0.098346, Valid Loss: 2.302510, Valid Acc: 0.098892, Time 00:00:49
Epoch 2. Train Loss: 2.266638, Train Acc: 0.128517, Valid Loss: 2.263254, Valid Acc: 0.165546, Time 00:00:39
Epoch 3. Train Loss: 1.932595, Train Acc: 0.243906, Valid Loss: 1.794690, Valid Acc: 0.303105, Time 00:00:39
Epoch 4. Train Loss: 1.692693, Train Acc: 0.342571, Valid Loss: 1.763555, Valid Acc: 0.355320, Time 00:00:39
Epoch 5. Train Loss: 1.473339, Train Acc: 0.445752, Valid Loss: 1.585759, Valid Acc: 0.438588, Time 00:00:39
Epoch 6. Train Loss: 1.262603, Train Acc: 0.538303, Valid Loss: 1.636224, Valid Acc: 0.454213, Time 00:00:39
Epoch 7. Train Loss: 1.067181, Train Acc: 0.616868, Valid Loss: 1.198231, Valid Acc: 0.568730, Time 00:00:39
Epoch 8. Train Loss: 0.886475, Train Acc: 0.684163, Valid Loss: 1.444764, Valid Acc: 0.552215, Time 00:00:39
Epoch 9. Train Loss